In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = pd.read_csv('home_prices.csv')

In [2]:
df.head()

Location  SquareFeet  Bedrooms  Bathrooms   Price
0   San Antonio        2833         4          3  419985
1       Houston        2743         3          1  361457
2     San Diego        2649         5          4  357192
3  Philadelphia        2867         3          2  418557
4       Phoenix        2596         2          4  446980

In [3]:
df = pd.get_dummies(df, columns=['Location'], drop_first=True)

In [4]:
df.head()

SquareFeet  Bedrooms  Bathrooms   Price  Location_Houston  \
0        2833         4          3  419985             False   
1        2743         3          1  361457              True   
2        2649         5          4  357192             False   
3        2867         3          2  418557             False   
4        2596         2          4  446980             False   

   Location_Los Angeles  Location_New York  Location_Philadelphia  \
0                 False              False                  False   
1                 False              False                  False   
2                 False              False                  False   
3                 False              False                   True   
4                 False              False                  False   

   Location_Phoenix  Location_San Antonio  Location_San Diego  
0             False                  True               False  
1             False                 False               False  
2             False                 False                True  
3             False                 False               False  
4              True                 False               False

In [5]:
# Features (X) and target variable (y)
X = df[['SquareFeet', 'Bedrooms', 'Bathrooms'] + list(df.columns[5:])]
y = df['Price']

In [6]:
X.head()

SquareFeet  Bedrooms  Bathrooms  Location_Los Angeles  Location_New York  \
0        2833         4          3                 False              False   
1        2743         3          1                 False              False   
2        2649         5          4                 False              False   
3        2867         3          2                 False              False   
4        2596         2          4                 False              False   

   Location_Philadelphia  Location_Phoenix  Location_San Antonio  \
0                  False             False                  True   
1                  False             False                 False   
2                  False             False                 False   
3                   True             False                 False   
4                  False              True                 False   

   Location_San Diego  
0               False  
1               False  
2                True  
3               False  
4               False

In [37]:
X.shape

(1000, 9)

In [7]:
y.head()

0    419985
1    361457
2    357192
3    418557
4    446980
Name: Price, dtype: int64

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [32]:
# Make predictions on the test set
predictions = model.predict(X_test)
predictions = predictions.astype(int)

In [35]:
# Calculate differences using list comprehension
differences = [predictions[i] - y_test.iloc[i] for i in range(len(predictions))]

# Create a DataFrame with predictions, actual values, and differences
results = pd.DataFrame({'predictions': predictions, 'actual': y_test.values, 'diff': differences})
results

predictions  actual    diff
0         555156  312858  242298
1         546563  518331   28232
2         548823  446694  102129
3         536208  650733 -114525
4         552253  371177  181076
..           ...     ...     ...
195       540619  331847  208772
196       544013  530914   13099
197       537265  761288 -224023
198       572626  460867  111759
199       566230  713452 -147222

[200 rows x 3 columns]

In [34]:
positive_count = (results['diff'] > 0).sum()
negative_count = (results['diff'] < 0).sum()
zero_count = (results['diff'] == 0).sum()

print(f'Positive Count: {positive_count}')
print(f'Negative Count: {negative_count}')
print(f'Zero Count: {zero_count}')

Positive Count: 104
Negative Count: 96
Zero Count: 0


In [36]:
import pickle
# Save the model to a pickle file
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [5]:
import pickle

# Load the model from the pickle file
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
# Make predictions on new data
predictions = loaded_model.predict([[2833,3,4,0,0,0,0,1,0]])

predictions = predictions.astype(int)

print(predictions[0])


562119


/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [1]:
import pickle
import pandas as pd

# Load the model from the pickle file
with open('model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# List of sample locations
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix',
             'Philadelphia', 'San Antonio', 'San Diego']

def predict_price(square_feet, bedrooms, bathrooms, location):
    location_flags = [0] * len(locations)
    if location in locations:
        location_flags[locations.index(location)] = 1
    else:
        print(f'Error: Invalid location - {location}')
        return None
    predicted_price = loaded_model.predict([[square_feet+bedrooms+bathrooms]+location_flags])
    predictions = predicted_price.astype(int)
    return predictions[0]


In [3]:
# Get the coefficients (weights) and intercept
coefficients = loaded_model.coef_  # Coefficients for each feature (square footage, bedrooms, bathrooms)
intercept = loaded_model.intercept_  # Intercept term

print("Coefficients:", coefficients)
print("Intercept:", intercept)

Coefficients: [-6.37542592e+00  1.22221821e+03  4.25294710e+03  4.50118216e+03
  1.82410398e+04  3.21225088e+03  4.65257369e+03  1.24667552e+04
 -3.87522794e+03]
Intercept: 547035.8941609525


In [19]:
def predict_price(square_feet, bedrooms, bathrooms, location):
    location_flags = [0] * len(locations)
    if location in locations:
        location_flags[locations.index(location)] = 1
    else:
        print(f'Error: Invalid location - {location}')
        return None
    arr=[square_feet]+[bedrooms]+[bathrooms]+location_flags
    coeff = [-6.37542592, 1222.21821, 4252.9471, 4501.18216, 18241.0398, 3212.25088, 4652.57369, 12466.7552, -3875.22794]
    for i in range(9):
        arr[i]=arr[i]*coeff[i]
    return sum(arr)+547035.8941609525

In [20]:

# Example usage
square_feet = 2833
bedrooms = 4
bathrooms = 3
location = 'San Diego'

predicted_price = predict_price(square_feet, bedrooms, bathrooms, location)
if predicted_price is not None:
    print(f'Predicted Price: {predicted_price}')

[2833, 4, 3, 0, 0, 0, 0, 0, 0, 0, 1]
Predicted Price: 546623.0266695925
